<a href="https://colab.research.google.com/github/shon-Rocky/jupyter-notebook/blob/main/PyTorch_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [41]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [42]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [43]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## Creating Models

In [44]:
class NeuralNetwork(nn.Module):
  def __init__(self) -> None:
     super().__init__()
     self.flatten = nn.Flatten()
     self.liner_relu_stack = nn.Sequential(
         nn.Linear(28*28, 512),
         nn.ReLU(),
         nn.Linear(512, 256),
         nn.ReLU(),
         nn.Linear(256, 10)# ouptut should be 10 because our data has 10 target
     )

  def forward(self, x):
        x = self.flatten(x)
        logits = self.liner_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (liner_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters

In [45]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [46]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # compute prediction

    pred = model(X)
    loss = loss_fn(pred, y)

    # Back propagation

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
        loss, current = loss.item(), (batch + 1) * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [49]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [50]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.181779  [   64/60000]
loss: 2.171009  [ 6464/60000]
loss: 2.111542  [12864/60000]
loss: 2.111809  [19264/60000]
loss: 2.085314  [25664/60000]
loss: 2.024649  [32064/60000]
loss: 2.043134  [38464/60000]
loss: 1.975408  [44864/60000]
loss: 1.988089  [51264/60000]
loss: 1.904521  [57664/60000]
Test Error: 
 Accuracy: 55.0%, Avg loss: 4.683289 

Epoch 2
-------------------------------
loss: 1.957507  [   64/60000]
loss: 1.928560  [ 6464/60000]
loss: 1.816245  [12864/60000]
loss: 1.827615  [19264/60000]
loss: 1.754884  [25664/60000]
loss: 1.702651  [32064/60000]
loss: 1.705331  [38464/60000]
loss: 1.616884  [44864/60000]
loss: 1.645984  [51264/60000]
loss: 1.525357  [57664/60000]
Test Error: 
 Accuracy: 57.8%, Avg loss: 3.795689 

Epoch 3
-------------------------------
loss: 1.630552  [   64/60000]
loss: 1.590526  [ 6464/60000]
loss: 1.441956  [12864/60000]
loss: 1.484720  [19264/60000]
loss: 1.392024  [25664/60000]
loss: 1.383216  [32064/600

## Saving Models

In [51]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading Models

In [52]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [53]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
